In [4]:
import os
import pandas as pd

In [5]:
data_path = os.path.expanduser('C:\\Users\\might\\Desktop\\jupyter notebooks\\u.data')

df = pd.read_csv(data_path, sep='\t', header=None)#the original df
df.head(5)

,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
print('total shape of df:', df.shape)

total shape of df: (100000, 4)


In [83]:
shuffled_df = df.loc[np.random.randint(0, 100000, size=df.shape[0])]
sm_train_df = shuffled_df[:80000]#small dataframe, trimming data points to random 2000 datapoints
test_df= shuffled_df[-20000:]
print('new trimmed dataset:', sm_train_df.shape, '\nnew test dataset:', test_df.shape)

new trimmed dataset: (80000, 4) 
new test dataset: (20000, 4)


In [87]:
#creating binary labels out of ratings
sm_train_df[2]= sm_train_df[2].apply(lambda x: int(x>3))
sm_df.head(5)

c:\users\might\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,0,1,2,3
75217,896,55,0,887157978
30449,536,230,1,882359779
85868,450,61,1,882376446
11983,223,22,1,891550649
46558,450,455,1,882376188


In [88]:
print('max value of user id:', max(sm_train_df[0]),'\nmax value of movie id:', max(sm_train_df[1]),'\nTherefore concatenated shape of input layer:',(max(sm_train_df[0])+max(sm_train_df[1]),))

max value of user id: 943 
max value of movie id: 1682 
Therefore concatenated shape of input layer: (2625,)


In [59]:
from keras.models import Model
from keras.layers import Input, Dense
from keras import metrics

In [89]:
user_in_layer = Input(shape=(max(sm_train_df[0]+1),))#top half of input layer
movie_in_layer = Input(shape=(max(sm_train_df[1]+1),))#bottom half of input layer

#total input moves as concatenated user_id & movie_id one-hot encoded forms

hid_layer_u = Dense(100)(user_in_layer)#latent dimension k=100 for user_id #_Embedding(100, )
hid_layer_m= Dense(100)(movie_in_layer)#latent dimension k=100 for movie_id

merge_layer = keras.layers.dot([hid_layer_u,hid_layer_m], axes=1)



predictions = Dense(1, activation='softmax')(merge_layer)

model = Model(inputs=[user_in_layer, movie_in_layer], outputs= predictions)

model.compile(loss= 'binary_crossentropy', optimizer='sgd', metrics= ['mae', 'accuracy'])

In [92]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 944)          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 1683)         0                                            
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 100)          94500       input_5[0][0]                    
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 100)          168400      input_6[0][0]                    
__________________________________________________________________________________________________
dot_3 (Dot

In [90]:
x_train_user_in = to_categorical(sm_train_df[0])#contains the one-hot encoded user_id data, shaped (batch_size, max(sm_df[0]))
x_train_movie_in = to_categorical(sm_train_df[1])#contains the one-hot encoded movie_id data, shaped (batch_size, max(sm_df[1]))

y_ratings= sm_train_df[2]
print('shape of x_train_user_in:', x_train_user_in.shape,'\nshape of x_train_movie_in:', x_train_movie_in.shape,
      '\nshape of x_train_user_in:',y_ratings.shape)

shape of x_train_user_in: (80000, 944) 
shape of x_train_movie_in: (80000, 1683) 
shape of x_train_user_in: (80000,)


In [91]:
model.fit([x_train_user_in,x_train_movie_in],y_ratings, epochs=100, batch_size=32)

Epoch 1/100
80000/80000 [==============================] - 5s 61us/step - loss: 15.9424 - mean_absolute_error: 1.0000 - acc: 0.0000e+00
Epoch 2/100
80000/80000 [==============================] - 5s 58us/step - loss: 15.9424 - mean_absolute_error: 1.0000 - acc: 0.0000e+00
Epoch 3/100
80000/80000 [==============================] - 5s 57us/step - loss: 15.9424 - mean_absolute_error: 1.0000 - acc: 0.0000e+00
Epoch 4/100
80000/80000 [==============================] - 5s 57us/step - loss: 15.9424 - mean_absolute_error: 1.0000 - acc: 0.0000e+00
Epoch 5/100
80000/80000 [==============================] - 5s 57us/step - loss: 15.9424 - mean_absolute_error: 1.0000 - acc: 0.0000e+00
Epoch 6/100
80000/80000 [==============================] - 5s 57us/step - loss: 15.9424 - mean_absolute_error: 1.0000 - acc: 0.0000e+00
Epoch 7/100
80000/80000 [==============================] - 5s 60us/step - loss: 15.9424 - mean_absolute_error: 1.0000 - acc: 0.0000e+00
Epoch 8/100
80000/80000 [=======================

KeyboardInterrupt: 

In [12]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
import numpy as np
model = Sequential()
model.add(Embedding(1000, 64, input_length=10))
# the model will take as input an integer matrix of size (batch, input_length).
# the largest integer (i.e. word index) in the input should be
# no larger than 999 (vocabulary size).
# now model.output_shape == (None, 10, 64), where None is the batch dimension.

input_array = np.random.randint(1000, size=(32, 10))

#model.compile('rmsprop', 'mse')
#output_array = model.predict(input_array)
#assert output_array.shape == (32, 10, 64)

In [13]:
model.input

<tf.Tensor 'embedding_2_input:0' shape=(?, 10) dtype=float32>

In [14]:
a= [5,8,15]

from keras.utils import to_categorical

In [16]:
b= to_categorical(a)

In [19]:
b[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)